In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
from imutils import paths
from cancernet import config
from cancernet.cancernet import CancerNet
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adagrad
from tensorflow.keras.callbacks import LearningRateScheduler
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib
matplotlib.use("Agg")
"""
    Similar Code is available here- https://pyimagesearch.com/2019/02/18/breast-cancer-classification-with-keras-and-deep-learning/
    DO CHECK THE DATASET EXTRACTION PROCESS
"""

'\n    Similar Code is available here- https://pyimagesearch.com/2019/02/18/breast-cancer-classification-with-keras-and-deep-learning/\n    DO CHECK THE DATASET EXTRACTION PROCESS\n'

In [2]:
NUM_EPOCHS = 10   #Use NUM_EPOCHS = 40 for better accuracy
INIT_LR = 1e-2
BS = 32

In [3]:
trainPaths = list(paths.list_images(config.TRAIN_PATH))
lenTrain = len(trainPaths)
lenVal = len(list(paths.list_images(config.VAL_PATH)))
lenTest = len(list(paths.list_images(config.TEST_PATH)))

trainLabels = [int(p.split(os.path.sep)[-2]) for p in trainPaths]

trainLabels = to_categorical(trainLabels)
classTotals = trainLabels.sum(axis=0)

#classWeight = classTotals.max()/classTotals
classWeight = dict()
# loop over all classes and calculate the class weight
for i in range(0, len(classTotals)):
    classWeight[i] = classTotals.max() / classTotals[i]

In [4]:

trainAug = ImageDataGenerator(
    rescale=1/255.0,
    rotation_range=20,
    zoom_range=0.05,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.05,
    horizontal_flip=True,
    vertical_flip=True,
    fill_mode="nearest")

valAug = ImageDataGenerator(rescale=1 / 255.0)

trainGen = trainAug.flow_from_directory(
    config.TRAIN_PATH,
    class_mode="categorical",
    target_size=(48, 48),
    color_mode="rgb",
    shuffle=True,
    batch_size=BS)
valGen = valAug.flow_from_directory(
    config.VAL_PATH,
    class_mode="categorical",
    target_size=(48, 48),
    color_mode="rgb",
    shuffle=False,
    batch_size=BS)
testGen = valAug.flow_from_directory(
    config.TEST_PATH,
    class_mode="categorical",
    target_size=(48, 48),
    color_mode="rgb",
    shuffle=False,
    batch_size=BS)

Found 199818 images belonging to 2 classes.
Found 22201 images belonging to 2 classes.
Found 55505 images belonging to 2 classes.


In [5]:
model = CancerNet.build(width=48, height=48, depth=3, classes=2)
opt = Adagrad(learning_rate=INIT_LR, decay=INIT_LR/NUM_EPOCHS)
model.compile(loss="binary_crossentropy", optimizer=opt, metrics=["accuracy"])

In [6]:
M = model.fit(
    trainGen,
    steps_per_epoch=lenTrain//BS,
    validation_data=valGen,
    validation_steps=lenVal//BS,
    class_weight=classWeight,
    epochs=NUM_EPOCHS)

Epoch 1/10
6244/6244 [==============================] - 362s 57ms/step - loss: 0.6281 - accuracy: 0.8133 - val_loss: 0.5173 - val_accuracy: 0.7904
Epoch 2/10
6244/6244 [==============================] - 339s 54ms/step - loss: 0.5897 - accuracy: 0.8239 - val_loss: 0.5122 - val_accuracy: 0.7919
Epoch 3/10
6244/6244 [==============================] - 332s 53ms/step - loss: 0.5827 - accuracy: 0.8263 - val_loss: 0.4916 - val_accuracy: 0.8023
Epoch 4/10
6244/6244 [==============================] - 349s 56ms/step - loss: 0.5824 - accuracy: 0.8253 - val_loss: 0.4988 - val_accuracy: 0.7964
Epoch 5/10
6244/6244 [==============================] - 360s 58ms/step - loss: 0.5792 - accuracy: 0.8263 - val_loss: 0.4967 - val_accuracy: 0.7972
Epoch 6/10
6244/6244 [==============================] - 331s 53ms/step - loss: 0.5789 - accuracy: 0.8273 - val_loss: 0.4907 - val_accuracy: 0.7985
Epoch 7/10
6244/6244 [==============================] - 347s 56ms/step - loss: 0.5781 - accuracy: 0.8264 - val_loss: 0

In [7]:
model.save('BreastCancer_model.h5', M)

In [8]:
print("Now evaluating the model")
testGen.reset()
pred_indices = model.predict(testGen, steps=(lenTest//BS)+1)

pred_indices = np.argmax(pred_indices, axis=1)

print(classification_report(testGen.classes, pred_indices,
                            target_names=testGen.class_indices.keys()))

cm = confusion_matrix(testGen.classes, pred_indices)
total = sum(sum(cm))
accuracy = (cm[0, 0]+cm[1, 1])/total
specificity = cm[1, 1]/(cm[1, 0]+cm[1, 1])
sensitivity = cm[0, 0]/(cm[0, 0]+cm[0, 1])
print(cm)
print(f'Accuracy: {accuracy}')
print(f'Specificity: {specificity}')
print(f'Sensitivity: {sensitivity}')

Now evaluating the model
1735/1735 [==============================] - 56s 32ms/step
              precision    recall  f1-score   support

           0       0.94      0.77      0.84     39736
           1       0.60      0.87      0.71     15769

    accuracy                           0.80     55505
   macro avg       0.77      0.82      0.77     55505
weighted avg       0.84      0.80      0.80     55505

[[30430  9306]
 [ 2067 13702]]
Accuracy: 0.7950995405819296
Specificity: 0.8689200329760923
Sensitivity: 0.7658043084356755


In [9]:
N = NUM_EPOCHS
plt.style.use("ggplot")
plt.figure()
plt.plot(np.arange(0, N), M.history["loss"], label="train_loss")
plt.plot(np.arange(0, N), M.history["val_loss"], label="val_loss")
plt.plot(np.arange(0, N), M.history["accuracy"], label="train_acc")
plt.plot(np.arange(0, N), M.history["val_accuracy"], label="val_acc")
plt.title("Training Loss and Accuracy on the IDC Dataset")
plt.xlabel("Epoch No.")
plt.ylabel("Loss/Accuracy")
plt.legend(loc="lower left")
plt.savefig('plot.png')
